# Part I : General questions

### a) Based on your gameplay experience, propose a simple A/B test that could deliver a quick win. What metrics would you monitor to evaluate its success?

✅ **Proposal 1 : Rewarded "Undo" Button** <br>

Offer 1 “Undo” per session for watching a rewarded video (RV). This can encourage ad engagement while reducing frustration from mistakes. Also make users get the habit of using power-ups so that they are more willing to pay in later levels.

✨ Metrics to monitor between control group and testing group:
* RV ads watched per DAU
* Session length
* Game completions rate
* Retention (Day 1/3)

✅ **Proposal 2 : “Continue Level” Discount** 

Offer a discounted coin price to retry failed levels, this is very important for levels higher than Level 60 since we have **Tower Streak**. Users might want to keep their level and not fall to fail the tower. This can encourage users to keeping trying and increase monetization.

✨ Metric to monitor between control group and testing group: 
* Retry Rate 
* Coin Spend 
* Retention Next Session

### b) If you were asked to brainstorm ways to improve the game’s long-term retention, and based on your experience playing it, what new feature or system would you propose adding to support this goal?

**Proposal 1 : 🤝 Social Layer : Friend Leaderboards + Social Sharing**

Allow friend invites,so friends can play together and see each others levels. Allow friends to send/receive “lives” or “undo tokens”. Add optional social logins with small rewards. Social connections dramatically boost retention. 

We can also send notifications like : 
* 🎉 “Your friend completed today’s puzzle!”
* 👥 Invite reward: free booster pack or undo.


**Proposal 2 : 🧱 Build Visual Map for Progression**

Build a **Visual Map** (like Candy Crush) showing level progress. Players can also collect **themed card decks** (for example :  "Jungle", "Galaxy") by playing. Include “milestone chests” every 10-20 levels. This can Adds long-term collection motivation and personalisation.

* 🎨 Unlock cosmetic decks or animations.
* 🏆 Progression bar = visible “journey.”

---


# Part II: Tracking Plan – “Race” & “Tower Streak” LiveOps Event

### a) What are the objectives of the two events according to you ?

🔵 For Tower Streak, the main goal is **retention and monetization**. It encourages daily play over multiple days (3-day format) which is good for retention. Once user fails one level, the event will end without reward, to maintain the win streak, users have higher chance to pay for power-ups which will boost IAP.

🟡 For Race, the main objective is to increase **engagement** and **extend session numbers and session length**. By creating a competition between 5 players to finish 30 levels fastest, users may stay longer in games to win it and play more sessions. The competition which builds tension and thrill can heighten **emotional investment**.

### b) Which events would you suggest implementing to evaluate the performance of the LiveOps events? Please describe each event, including:
* **When the event should be triggered**
* **What parameters it should include (specific parameters)**

I would suggest 2 events too, they are **liveops_event_session and liveops_event_progress**

🔷 1. liveops_event_session

📍When to Trigger: At the start of a LiveOps event (Tower or Race) OR when the player resumes an event session.

| Parameter            | Description                  |
| -------------------- | ---------------------------- |
| `user_id`            | Player ID                    |
| `event_name`         | `"tower_streak"` or `"race"` |
| `event_day`          | Day within event cycle       |
| `session_start_time` | Timestamp                    |
| `entry_type`         | `"new"` or `"resume"`        |
| `player_level`       | Current game level           |
| `coin_balance`       | Coin count at entry          |
| `is_first_time`      | Boolean for first-time entry |
| `boosters_available` | Booster inventory snapshot   |

🔶 2. liveops_event_progress

📍When to Trigger: After each match attempt within the event. Also triggered at event exit or reward claim.

| Parameter          | Description                                       |
| ------------------ | ------------------------------------------------- |
| `user_id`          | Player ID                                         |
| `event_name`       | `"tower_streak"` or `"race"`                      |
| `attempt_id`       | Unique attempt reference                          |
| `level`            | Game level attempted                              |
| `result`           | `"win"` / `"loss"` / `"quit"`                     |
| `time_spent`       | Duration of attempt                               |
| `boosters_used`    | Number used in attempt                            |
| `reward_earned`    | Coins or items gained (if any)                    |
| `final_rank`       | For Race: 1–5                           |
| `streak_count`     | For Tower: win streak count                       |
| `event_outcome`    | `"completed"`, `"lost"`, `"dnf"` (did not finish) |
| `session_end_time` | Timestamp                                         |

### c) Based on these events, which key metrics would you compute to measure the features’ performance?

🔷 Metrics from liveops_event_session (Session-Level Insights)


| **Metric**                        | **What it Tells You**                              | **How to Compute**                                                  |
| --------------------------------- | -------------------------------------------------- | ------------------------------------------------------------------- |
| **Participation Rate**            | % of eligible players who start the event          | `distinct(user_id)` in `liveops_event_session / eligible_players` |                     |
| **Daily Active Participants**     | Daily event engagement                             | Count of `user_id` per `event_day`                                  |                                              |
| **Avg Coin Balance on Entry**     | Monetization readiness before participating        | `avg(coin_balance)`                                                 |
| **Booster Ownership on Entry**    | % of users with available boosters                 | % with `boosters_available > 0`                                     |





--

🔶 Metrics from liveops_event_progress (Match & Outcome Insights)
| **Metric**                         | **What it Tells You**                       | **How to Compute**                                      |
| ---------------------------------- | ------------------------------------------- | ------------------------------------------------------- |
| **Attempt Volume**                 | Total number of games played in the event   | Count of `attempt_id`                                   |
| **Completion Rate (Race)**         | % of users who completed 30 levels          | % where `event_outcome = 'completed'`                   |
| **Streak Failure Rate (Tower)**    | % of users who lose before reaching goal    | % where `event_outcome = 'lost'`                        |
| **Avg Win Streak (Tower)**         | Skill performance of users                  | `avg(streak_count)` where `event_name = 'tower_streak'` |
| **Avg Match Duration**             | Time spent per level                        | `avg(time_spent)`                                       |
| **Booster Usage Rate**             | Engagement with power-ups                   | % where `boosters_used > 0`                             |
| **Reward Claim Rate**              | % of players who earned and claimed rewards | % where `reward_earned > 0`                             |


These metrics help to measures in-event behavior, success/failure rates, outcomes, and monetization signals.
We can compare LiveOps vs. Non-LiveOps Session to see if this event increase playtime, session frequency, or revenue. Then Compare D1/D3/D7 retention between participants and non-participants can help to check if this event uplift event retention.

| **Metric**                     | **What it Tells You**                       | **How to Compute**                                                |
|--------------------------------|---------------------------------------------|-------------------------------------------------------------------|
| **Participation Rate**         | % of eligible players who start the event   | `distinct(user_id)` in `liveops_event_session / eligible_players` |
| **Daily Active Participants**  | Daily event engagement                      | Count of `user_id` per `event_day`                                |
| **Avg Coin Balance on Entry**  | Monetization readiness before participating | `avg(coin_balance)`                                               |
| **Booster Ownership on Entry** | % of users with available boosters          | % with `boosters_available > 0`                                   |
